## Data convertor for Revenue and Marketing planning
Data convertor loads the data from the simulation files and transforms them into files that can be loaded directly to the Revenue Planning and Marketing planning systems

In [55]:
import pandas as pd
import numpy as np
import xlrd
pd.options.mode.chained_assignment = None  # default='warn' #to switch the wornings off

#PATHS to the files
path_sf_off_premise = "B004_OFF_PREMISE_SIM_FILE_20201207.xlsx"
path_sf_on_premise = "B004_ON_PREMISE_SIM_20201204.xlsx"
path_lookup_file = "lookup_B001.xlsx" #is still needed?? - could be possibly replaced by the lookup inside of the sf file? transfer of responsibility

months = 12 #fill the number of months that are planned exmpl: RE 9+3 --> 3 months
year = "BP 2021" #fill as in Simulation file, data filtered by this value / Version later
cz = "18" #starting numbers in CPLs / country split done based on this
sk = "19" #starting numbers in CPLs / country split done based on this

#Dictionary
conditions = {"on_invoice":"YAA%", "not_assigned":"#", "promo_leaflet":"ZEFQ", "promo":"ZBCQ", "1.75":"062"}

### Files load into Pandas
Code loads the data from the simulation files for further transformation

Excel files MUST be CLOSED in order for the next code to work!

In [56]:
#ON INVOICE data
on_invoice_data_OFP = pd.read_excel(path_sf_off_premise, sheet_name="Pivot_Adam", usecols="A:G,AF:AQ", skiprows=2, convertors={4:(lambda x: str(x)),1:str,2:str,3:str,5:(lambda x: str(x)),6:str})
on_invoice_data_ONP = pd.read_excel(path_sf_on_premise, sheet_name="Pivot_Adam", usecols="A:S", skiprows=2)

#OFF INVOICE data
off_invoice_data_OFP = pd.read_excel(path_sf_off_premise, sheet_name="Off inv dis", usecols="D,F,H:AE", skiprows=0)
off_invoice_data_ONP = pd.read_excel(path_sf_on_premise, sheet_name="Off inv. Disc.", usecols="H,J,L:AI", skiprows=0)

#PROMO DATA
promo_data_OFP = pd.read_excel(path_sf_off_premise, sheet_name="Pivot_Adam", usecols="A:AE", skiprows=2)
promo_ls = pd.read_excel(path_sf_off_premise, sheet_name="data", usecols="B,E,J:O,BT:CE", skiprows=90)

#LOOKUP DATA
lookup_data = pd.read_excel(path_lookup_file, sheet_name="lookup", usecols="A:R")

### Functions used

In [57]:
#fce adjusting the data loaded - excel load loads strings as numbers and ommits zeros in the beginning of the key
def adjust(x):
    if pd.isnull(x):
        return x
    elif len(str(int(float(x)))) < 3:
        return (3-len(str(int(float(x))))) * "0" + str(int(float(x)))
    else:
        return str(int(float(x)))

### On invoice Discount Transformation

Following contains data transformation into an excel file that can be loaded into the system for the condition planning in Revenue Planning System

In [58]:
#on invoice off premise data preparation
on_invoice_data_OFP.columns = ["cpl_no","pack_type_no","pack_size_no","sales_unit_no","item_category_no","brand_no","flavor_no", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"]
on_invoice_data_OFP["condition"] = conditions["on_invoice"]
on_invoice_data_OFP[["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"]] = round(on_invoice_data_OFP[["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"]]*100,2)
on_invoice_data_OFP = on_invoice_data_OFP[["cpl_no","condition","pack_type_no","pack_size_no","sales_unit_no","item_category_no","brand_no","flavor_no", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"]]

In [59]:
#on invoice on premise data preparation - Correct column names, percentages multiplied by 100, correct condition added, rearranging the columns in a right order
on_invoice_data_ONP.columns = ["cpl_no","pack_type_no","pack_size_no","sales_unit_no","item_category_no","brand_no","flavor_no", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"]
on_invoice_data_ONP["condition"] = conditions["on_invoice"]
on_invoice_data_ONP[["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"]] = round(on_invoice_data_ONP[["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"]]*100,2)
on_invoice_data_ONP = on_invoice_data_ONP[["cpl_no","condition","pack_type_no","pack_size_no","sales_unit_no","item_category_no","brand_no","flavor_no", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"]]

In [60]:
#Adding OFPremise and ONPremise data together for the further wrangling
on_invoice_data = pd.concat([on_invoice_data_OFP,on_invoice_data_ONP],axis=0,ignore_index=True)


In [61]:
#Prolly unnecessary - should be correct in an excel file
#on_invoice_codes["item_category_no"] = on_invoice_codes["item_category_no"].fillna(0).astype(int)
#on_invoice_codes["cpl_no"] = on_invoice_codes.cpl_no.astype(int)

#Flavor je # if not spirit, spirits left alone / already in the lookup
#replacing "#"" with nan, so that its empty in final file
on_invoice_data["flavor_no"] = on_invoice_data.flavor_no.replace("#",np.nan)

#dropping months I dont need
on_invoice_months = on_invoice_data.iloc[:,np.r_[0:8,(len(on_invoice_data.columns)-months):len(on_invoice_data.columns)]]
# puvodni on_invoice_months = on_invoice_data_OFP.iloc[:,np.r_[0:8,(len(on_invoice_data.columns)-months):len(on_invoice_data.columns)]]

#Dropping row where values in all planned months are zeros
on_invoice_zeros = on_invoice_months[(on_invoice_months.iloc[:,(-1*months):-1].sum(axis=1))>0]

#DROPPING DUPLICATES - all columns duplicated
on_invoice_duplicates = on_invoice_zeros.drop_duplicates(subset=on_invoice_zeros.columns, keep="first")
on_invoice_final = on_invoice_duplicates


### OFF INVOICE Discount Transformation

Following contains data transformation into an excel file that can be loaded into the system for the condition planning in Revenue Planning System

In [62]:
#OFF PREMISE renaming columns, rounding %
off_invoice_data_OFP.columns = ["cpl_no","condition","item_category_no","item_category","brand_no","brand","flavor_no","flavor","pack_type_no","pack_type","pack_size_no","pack_size","sales_unit_no","sales_unit","1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"]
off_invoice_data_OFP[["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"]] = round(off_invoice_data_OFP[["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"]],2)

#ON PREMISE renaming columns, rounding %
off_invoice_data_ONP.columns = ["cpl_no","condition","item_category_no","item_category","brand_no","brand","flavor_no","flavor","pack_type_no","pack_type","pack_size_no","pack_size","sales_unit_no","sales_unit","1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"]
off_invoice_data_ONP[["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"]] = round(off_invoice_data_ONP[["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"]],2)

#OFF INVOICE concat OFF PREMISE & ON PREMISE
off_invoice_data = pd.concat([off_invoice_data_OFP,off_invoice_data_ONP],axis=0,ignore_index=True)

In [63]:
#changing all CPLs into integers / first convert nans to 0
off_invoice_data["cpl_no"] = off_invoice_data.cpl_no.fillna(0).astype(int)

#Specifiing the columns used
off_invoice_codes = off_invoice_data[["cpl_no","condition","pack_type_no","pack_size_no","sales_unit_no","item_category_no","brand_no","flavor_no","1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"]]

#dropping the months I dont need
off_invoice_months = off_invoice_codes.iloc[:,np.r_[0:8,(len(off_invoice_codes.columns)-months):len(off_invoice_codes.columns)]]

#Dropping zeros QUESTIONABLE!!!!!
off_invoice_zeros = off_invoice_months[(off_invoice_months.iloc[:,(-1*months):-1].sum(axis=1))>0]

#Dropping duplicates
off_invoice_duplicates = off_invoice_zeros.drop_duplicates(subset=off_invoice_zeros.columns, keep="first")

#Dropping duplicated characteristics
added_months_off = off_invoice_duplicates.iloc[:,-months:]
off_invoice_duplicates["SUM"] = added_months_off.sum(axis=1)
off_invoice_spirits = off_invoice_duplicates.sort_values("SUM").drop_duplicates(subset=off_invoice_duplicates.columns[:8], keep="last")

#Dropping SUM
off_invoice_final = off_invoice_spirits.drop("SUM", axis=1)

#repacing # with nans
off_invoice_final = off_invoice_final.replace("#",np.nan)

### REGULAR CONDITIONS (off inv & on inv)
Following will concat ON INVOICE and OFF INVOICE conditions, melt them in the right format, split them by country and export them as csv for the upload into the REVENU PLANNING System

In [64]:
#concat
regular = pd.concat([on_invoice_final, off_invoice_final], axis=0, ignore_index=True)

#Applying adjust function on pack_size_no and sales_unit_no - incorrectly transformed into int and float, omitting 0, corrcting:
regular['pack_size_no'] = regular.pack_size_no.apply(adjust)
regular['sales_unit_no'] = regular.sales_unit_no.apply(adjust)

#melting the data = unpivot + adjusting the formats, comma in the final format
regular_melted = pd.melt(regular.iloc[:,:], id_vars=regular.columns[:8])
regular_melted["cpl_no"] = regular_melted.cpl_no.astype(str)
regular_melted["value"] = regular_melted.value.astype(str)
regular_melted["value"] = regular_melted["value"].str.replace(".",",")

In [65]:
#CZ SK split of melted data
cz_regular_output = regular_melted[regular_melted["cpl_no"].str.startswith(cz)]
sk_regular_output = regular_melted[regular_melted["cpl_no"].str.startswith(sk)]

In [66]:
#To load file
cz_regular_output.to_csv("cz_regular_load.txt", sep='\t', index=False)
sk_regular_output.to_csv("sk_regular_load.txt", sep='\t', index=False)

### PROMO Discount Transformation

Following contains data transformation into an excel file that can be loaded into the system for the condition planning in Revenue Planning System

In [67]:
#RENAMING columns, version according to SF, correct order of columns, % multiplied by 100
promo_data_OFP.columns = ["cpl_no","pack_type_no","pack_size_no","sales_unit_no","item_category_no","brand_no","flavor_no","1_p", "2_p", "3_p", "4_p", "5_p", "6_p", "7_p", "8_p", "9_p", "10_p", "11_p", "12_p", "1_d", "2_d", "3_d", "4_d", "5_d", "6_d", "7_d", "8_d", "9_d", "10_d", "11_d", "12_d"]
promo_data_OFP[["1_p", "2_p", "3_p", "4_p", "5_p", "6_p", "7_p", "8_p", "9_p", "10_p", "11_p", "12_p", "1_d", "2_d", "3_d", "4_d", "5_d", "6_d", "7_d", "8_d", "9_d", "10_d", "11_d", "12_d"]] = round(promo_data_OFP[["1_p", "2_p", "3_p", "4_p", "5_p", "6_p", "7_p", "8_p", "9_p", "10_p", "11_p", "12_p", "1_d", "2_d", "3_d", "4_d", "5_d", "6_d", "7_d", "8_d", "9_d", "10_d", "11_d", "12_d"]]*100,2)

#item cat to INT (no decimals), cpl_no to INT, replace # with nans
promo_data_OFP["item_category_no"] = promo_data_OFP.item_category_no.fillna(0).astype(int)
promo_data_OFP["cpl_no"] = promo_data_OFP.cpl_no.astype(int)
promo_data_OFP["flavor_no"] = promo_data_OFP.flavor_no.replace("#", np.nan)

#Applying adjust function on pack_size_no and sales_unit_no - incorrectly transformed into int and float, omitting 0, corrcting:
promo_data_OFP['pack_size_no'] = promo_data_OFP.pack_size_no.apply(adjust)
promo_data_OFP['sales_unit_no'] = promo_data_OFP.sales_unit_no.apply(adjust)

#Dropping months not needed
promo_months = promo_data_OFP.iloc[:,np.r_[0:7,(len(promo_data_OFP.columns)-(months+12)):(len(promo_data_OFP.columns)-12),(len(promo_data_OFP.columns)-(months)):(len(promo_data_OFP.columns))]]

#Dropping rows that have only 0 values in the months I plan for
promo_zeros = promo_months[(promo_months.iloc[:,(-1*months*3):(-1*months*2)].sum(axis=1)>0)&(promo_months.iloc[:,(-1*months*2):(-1*months)].sum(axis=1)>0)]

#DROPPING DUPLICATES - all columns duplicated
promo_duplicates = promo_zeros.drop_duplicates(subset=promo_zeros.columns, keep="first")

#addind index column full of nans
promo_duplicates["index"] = np.nan

#rearranging the columns
promo_final = promo_duplicates[["index","cpl_no","brand_no","flavor_no","item_category_no","pack_type_no","pack_size_no","sales_unit_no","1_p", "2_p", "3_p", "4_p", "5_p", "6_p", "7_p", "8_p", "9_p", "10_p", "11_p", "12_p", "1_d", "2_d", "3_d", "4_d", "5_d", "6_d", "7_d", "8_d", "9_d", "10_d", "11_d", "12_d"]]

In [68]:
#SPLITTING into two dfs -> 1. pressure, 2. depth - se it can be melted separately and then merged
promo_final_p = promo_final.iloc[:,:-months]
promo_final_d = promo_final.iloc[:,np.r_[0:8,(len(promo_final.columns)-months):len(promo_final.columns)]]

#Renaming the columns
promo_final_p = promo_final_p.rename(columns={"1_p":"1","2_p":"2",'3_p':"3",'4_p':"4",'5_p':"5",'6_p':"6",'7_p':"7",'8_p':"8",'9_p':"9",'10_p':"10",'11_p':"11",'12_p':"12"})
promo_final_d = promo_final_d.rename(columns={"1_d":"1","2_d":"2",'3_d':"3",'4_d':"4",'5_d':"5",'6_d':"6",'7_d':"7",'8_d':"8",'9_d':"9",'10_d':"10",'11_d':"11",'12_d':"12"})

In [69]:
#Melting the data (unpivotting) - separately p and d and then merging them, so I have desired output - month, p, d
promo_p_melted = pd.melt(promo_final_p, id_vars=promo_final_p.columns[:8])
promo_d_melted = pd.melt(promo_final_d, id_vars=promo_final_d.columns[:8])

#OPRAVENO d a p p5ehoyeno, bylo naruby
promo_melted = pd.merge(promo_d_melted, promo_p_melted, on=["index","cpl_no","brand_no","flavor_no","item_category_no","pack_type_no","pack_size_no","sales_unit_no","variable"], how="left")

#Adjusting fromats of certain columns, replacing . with comma as planning system demands
promo_melted["cpl_no"] = promo_melted.cpl_no.astype(str)
promo_melted["variable"] = promo_melted.variable.astype(str)

#dopln2no zne aby nebyly nuly
#dropping zeros if there are yeros in either of the value - value_x (depth), value_y (pressure)
promo_melted = promo_melted[(promo_melted["value_y"]>0)]
promo_melted = promo_melted[(promo_melted["value_x"]>0)]

#pressure adjustments
promo_melted["value_x"] = promo_melted.value_x.replace(np.nan,0)
promo_melted["value_x"] = promo_melted.value_x.astype(str)
promo_melted["value_x"] = promo_melted["value_x"].str.replace(".",",")

#depth adjustments
promo_melted["value_y"] = promo_melted.value_y.replace(np.nan,0)
promo_melted["value_y"] = promo_melted.value_y.astype(str)
promo_melted["value_y"] = promo_melted["value_y"].str.replace(".",",")

In [70]:
#CZ SK split of melted data
cz_promo_output = promo_melted[promo_melted["cpl_no"].str.startswith(cz)]
sk_promo_output = promo_melted[promo_melted["cpl_no"].str.startswith(sk)]

In [71]:
cz_promo_output.to_csv("cz_promo_load.txt", sep='\t', index=False)
sk_promo_output.to_csv("sk_promo_load.txt", sep='\t', index=False)

### LUMP SUMS Discount Transformation

Following contains data transformation into an excel file that can be loaded into the system for the condition planning in Marketing Planning System

In [72]:
#PROMO LS renaming columns, choosing only correct "year" (version) & rearranging columns, changing formats of %
promo_ls.columns = ["version","cpl_no","pack_type_no","pack_size_no","sales_unit_no","item_category_no","brand_no","flavor_no", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"]
#Filtering only corect version from SF
promo_ls = promo_ls[promo_ls["version"] == year]

# #adding columns for index and condition
# promo_ls['condition'] = conditions["promo_leaflet"]
# promo_ls["index"] = np.nan

In [73]:
#Rearragnging columns
#promo_ls = promo_ls[['index',"cpl_no",'condition',"item_category_no","brand_no","flavor_no","pack_type_no","pack_size_no","sales_unit_no","1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"]]
promo_ls["cpl_no"] = promo_ls.cpl_no.astype(int)
promo_ls["cpl_no"] = promo_ls.cpl_no.astype(str)

In [74]:
#CREATING THE AMOUNTS
#Groupby
promo_pregrouped = promo_ls[["cpl_no","1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"]]
promo_ls_grouped = promo_pregrouped.groupby("cpl_no").sum()
promo_ls_grouped = promo_ls_grouped[promo_ls_grouped.iloc[:,-months:].sum(axis=1)>0]

#index to column = po groupby it moves to index
promo_ls_grouped.reset_index(inplace=True)

#adding the condition and empty index
promo_ls_grouped["condition"] = conditions["promo_leaflet"]
promo_ls_grouped["index"] = np.nan

#Rearragning, datatype adjstmnt
promo_ls_grouped = promo_ls_grouped[["index","cpl_no","condition","1","2","3","4","5","6","7","8","9","10","11","12"]]
promo_ls_grouped[["1","2","3","4","5","6","7","8","9","10","11","12"]] = promo_ls_grouped[["1","2","3","4","5","6","7","8","9","10","11","12"]].astype(int)


In [75]:
#unpivot
promo_ls_melted = pd.melt(promo_ls_grouped, id_vars=promo_ls_grouped.columns[:3])

#exclude zeros in the output
promo_ls_final = promo_ls_melted[promo_ls_melted["value"]!=0]

In [76]:
#CREATING THE ALOCATION RULES
#USING THE LOOKUP - possible start with this and create alo first and then the amounts - simplified process
promo_ls_alo = promo_ls[["cpl_no","item_category_no","brand_no","flavor_no","pack_type_no","pack_size_no","sales_unit_no","1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"]]
promo_ls_alo["index"] = np.nan
promo_ls_alo["condition"] = conditions["promo_leaflet"]

#Data straight form sheet data, so flavors are present, not #, necessary to replace by nans except for spirits
promo_ls_alo.loc[(promo_ls_alo.item_category_no!=2145), "flavor_no"] = np.nan

#Rearranging and dropping columns
promo_ls_alo = promo_ls_alo[["index","cpl_no","condition","item_category_no","brand_no","flavor_no","pack_type_no","pack_size_no","sales_unit_no","1","2","3","4","5","6","7","8","9","10","11","12"]]

#only non zeros
promo_ls_alo = promo_ls_alo[promo_ls_alo.iloc[:,-months:].sum(axis=1)>0]

#adding the allocation rule
promo_ls_alo["selection"] = 1
promo_ls_alo_final = promo_ls_alo[["index","cpl_no","condition","item_category_no","brand_no","flavor_no","pack_type_no","pack_size_no","sales_unit_no","selection"]]

#item cat as a float
promo_ls_alo_final["item_category_no"] = promo_ls_alo_final.item_category_no.astype(int)

#Applying adjust function on pack_size_no and sales_unit_no - incorrectly transformed into int and float, omitting 0, corrcting:
promo_ls_alo_final['pack_size_no'] = promo_ls_alo_final.pack_size_no.apply(adjust)
promo_ls_alo_final['sales_unit_no'] = promo_ls_alo_final.sales_unit_no.apply(adjust)

#dropping duplicates
promo_ls_alo_final = promo_ls_alo_final.drop_duplicates(subset=promo_ls_alo_final.columns,keep="first")



In [77]:
cz_promo_ls_output = promo_ls_final[promo_ls_final["cpl_no"].str.startswith(cz)]
sk_promo_ls_output = promo_ls_final[promo_ls_final["cpl_no"].str.startswith(sk)]

cz_promo_ls_alo_output = promo_ls_alo_final[promo_ls_alo_final["cpl_no"].str.startswith(cz)]
sk_promo_ls_alo_output = promo_ls_alo_final[promo_ls_alo_final["cpl_no"].str.startswith(sk)]

In [78]:
#exporting Pormo LS allocation rules
cz_promo_ls_alo_output.to_csv("cz_promo_ls_alo_load.txt", sep='\t', index=False)
sk_promo_ls_alo_output.to_csv("sk_promo_ls_alo_load.txt", sep='\t', index=False)

#exporting Promo LS
cz_promo_ls_output.to_csv("cz_promo_ls_load.txt", sep='\t', index=False)
sk_promo_ls_output.to_csv("sk_promo_ls_load.txt", sep='\t', index=False)

### CHECKS REGULAR Discounts
What is missing, is the data MINT?

add check for subzero values in reaw data - there should be none!

check all the places where I use illc with months






1. po natáhnutí bych u promo neměl mít ve sloupcích ´j-o nikde NA hodnoty - tzn dej mi hodnoty z promo_ls, kde jsou NAcka - musí být empty
2. po natáhnutí bych nikde u on invoice alokace podle lookup adam neměl mít prázdné nebo NA hodnoty - dej mi regular final, filtrováno pro on invoice, kde jsou na
3. regular final - podmínka, která bude mít on invoice %YDA (nevím) by neměla mít nikde NA u alokace
4. testovat pro menší počet měsíců
5. vyhodit celkovou sumu promo LS leafletů
6. Vytáhnout záporné hodnoty slev




In [ ]:
#testing if there is a wrong allocation of SAP codes - first couple of columns
promo_test_1 = promo_data_OFP[(promo_data_OFP[] == "#N/A")
                             or (promo_data_OFP[] == "#NA")
                             or (promo_data_OFP[] == "#NA")
                             or (promo_data_OFP[] == "#NA")
                             or (promo_data_OFP[] == "#NA")]

#testing if there are negative values in promo depth = wrong plannning of promo prices in the sim file
promo_test_2 = promo_data_OFP[(promo_data_OFP[] < 0)
                             or (promo_data_OFP[] < 0)
                             or (promo_data_OFP[] < 0)
                             or (promo_data_OFP[] < 0)
                             or (promo_data_OFP[] < 0)]

In [ ]:
#testing if there are negative values in on invoice discounts = wrong plannning of on invoice prices in the sim file
promo_test_2 = promo_data_OFP[(promo_data_OFP[] < 0)
                             or (promo_data_OFP[] < 0)
                             or (promo_data_OFP[] < 0)
                             or (promo_data_OFP[] < 0)
                             or (promo_data_OFP[] < 0)]

In [13]:
#DOLADIT ROBUST TESTING
#-are there values with NA?
#-are there values that.....


#Check --> code can be used to show ON IVOICES that didnt connect in lookup file
test_1 = regular[(regular["condition"]=="YAA%")&(regular["sales_unit_no"].isnull())]
test_on_invoice_OFP = on_invoice_data_merged[on_invoice_data_merged["sales_unit_no"].isnull()]
#test_on_invoice_ONP
#test_certain CPL OFF INVOICE
test_2 = regular[(regular["condition"]=="ZDA%")
                 &(regular["cpl_no"]==199210)
                 #&(regular["pack_size_no"]!=conditions["1.75"])
                ]

#test OFF INVOICE RAW data
test_3 = off_invoice_data[(off_invoice_data["condition"]=="ZDA%")
                 &(off_invoice_data["cpl_no"]==199210)
                 #&(regular["pack_size_no"]!=conditions["1.75"])
                ]

test_4 = sk_regular_output[(sk_regular_output["condition"]=="ZDA%")
                 &(sk_regular_output["cpl_no"]=="199210")
                 #&(regular["pack_size_no"]!=conditions["1.75"])
                ]
test_4
#sk_regular_output.info()


,cpl_no,condition,pack_type_no,pack_size_no,sales_unit_no,item_category_no,brand_no,flavor_no,variable,value
4197,199210,ZDA%,NaN,NaN,NaN,NaN,NaN,NaN,1,"17,03"
8419,199210,ZDA%,NaN,NaN,NaN,NaN,NaN,NaN,2,"17,03"
12641,199210,ZDA%,NaN,NaN,NaN,NaN,NaN,NaN,3,"17,03"
16863,199210,ZDA%,NaN,NaN,NaN,NaN,NaN,NaN,4,"17,03"
21085,199210,ZDA%,NaN,NaN,NaN,NaN,NaN,NaN,5,"17,03"
25307,199210,ZDA%,NaN,NaN,NaN,NaN,NaN,NaN,6,"17,03"
29529,199210,ZDA%,NaN,NaN,NaN,NaN,NaN,NaN,7,"17,03"
33751,199210,ZDA%,NaN,NaN,NaN,NaN,NaN,NaN,8,"17,03"
37973,199210,ZDA%,NaN,NaN,NaN,NaN,NaN,NaN,9,"17,03"
42195,199210,ZDA%,NaN,NaN,NaN,NaN,NaN,NaN,10,"17,03"




### Zmeny oproti minule verzi

1. smazan prvni radek v Pivot_Adam, abych mohl data rovnou stahovat
2. document on premise - smazana data za Pressure - v pivot Ada
3. ON Premise - Pivot Adam - Remove grand totals from the pivot

### Kontrola
1. jsou off premise nasobeny spravnym cislem aby dala procenta?

In [79]:
#shows duplicates / must be no duplicates - indicates mistake in simfiles
sk_regular_output[sk_regular_output.duplicated(subset=cz_regular_output.columns[:9], keep=False)]

,cpl_no,condition,pack_type_no,pack_size_no,sales_unit_no,item_category_no,brand_no,flavor_no,variable,value


In [ ]:
#nulov0 values v regular - ybavit se jich

In [80]:
cz_regular_output[cz_regular_output.duplicated(subset=cz_regular_output.columns[:9], keep=False)]

,cpl_no,condition,pack_type_no,pack_size_no,sales_unit_no,item_category_no,brand_no,flavor_no,variable,value
